In [ ]:
## Load Folder Drive
import os, sys
from google.colab import drive
drive.mount('/content/drive/')

## Move Pointer File ke DATA_UM
%cd /content/drive/MyDrive/DATA_UM

## Declare Root Project File
root_file = "/content/drive/MyDrive/DATA_UM"

Mounted at /content/drive/
/content/drive/MyDrive/DATA_UM


In [ ]:
## Install necessary libraries
# &> /dev/null = Remove output Install
!pip install transformers &> /dev/null

# Import necessary libraries
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch

import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap

import re
import string

import os.path
from os import path

from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

rcParams['figure.figsize'] = 12, 8

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
## Declare Hashtag Data
hashtag = "ultramilkxstraykids"

In [ ]:
## Declare Bert Model and Load Tokenizer + Bert Untrained.

from transformers import AutoTokenizer, AutoModel

# Ganti ini kalau mau ganti model basennya.
MODEL_NAME = 'indolem/indobert-base-uncased'

# Load Token sama Base Model.
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
bert_model = AutoModel.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/234k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
# Build the Sentiment Classifier class
class SentimentClassifier(nn.Module):

    # Constructor class
    def __init__(self, n_classes):
        super(SentimentClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(MODEL_NAME)
        self.drop = nn.Dropout(p=0.3)
        self.out = nn.Linear(self.bert.config.hidden_size, n_classes)

    # Forward propagaion class
    def forward(self, input_ids, attention_mask):
        _, pooled_output = self.bert(
          input_ids=input_ids,
          attention_mask=attention_mask,
          return_dict=False
        )
        #  Add a dropout layer
        output = self.drop(pooled_output)
        return self.out(output)

In [ ]:
## Declare Class Sentiment.
class_names = ['Negative', 'Neutral', 'Positive']

## Declare Max Length Token based On Report Token.
MAX_LEN = 75

In [ ]:
# Used Parameter Training
EPOCHS = 3
LR = 2e-5
BATCH_SIZE = 16

# Instantiate the model and move to classifier
model = SentimentClassifier(len(class_names))

# IF Load with GPU
model.load_state_dict(torch.load('model/ultramilk_initialize/bert_model_e5_lr2e-05_bs32.pkl', map_location=torch.device('cpu')))

# IF Load with CPU
#model.load_state_dict(torch.load('model/'+hashtag+'/bert_model_e'+str(EPOCHS)+'_lr'+str(LR)+'_bs'+str(BATCH_SIZE)+'.pkl', map_location=torch.device('cpu')))

# Create Object Model
model = model.to(device)

In [ ]:
## Load Preprocess SLANG Module

# Load Dataframe
df_tmp_nasalsabila = pd.read_csv('data/slang/colloquial-indonesian-lexicon_nasalsabila.csv')
df_slang_fendiirfan = pd.read_csv('data/slang/colloquial-indonesian-lexicon_fendiirfan.csv')
df_slang_custom = pd.read_csv('data/slang/custom_dictonary_slang.csv')
df_slang_custom_exception = pd.read_csv('data/slang/custom_dictonary_slang_dont_swap.csv')

# Transform and Delete Duplicate Data from nasalsabila
df_slang_nasalsabila = df_tmp_nasalsabila[['slang', 'formal']]
df_slang_nasalsabila.drop_duplicates(inplace=True)

# Transform and Delete Duplicate Data from fendiirfan
df_slang_fendiirfan.dropna(inplace=True)
df_slang_fendiirfan.rename(columns={'kataAlay': 'slang', 'kataBaik': 'formal'}, inplace=True)
df_slang_fendiirfan.drop_duplicates(inplace=True)

# Transform and Delete Duplicate our own data.
df_slang_custom.drop_duplicates(inplace=True)
df_slang_custom_exception.drop_duplicates(inplace=True)

def remove_slang(text, df_tmp):
    # To Lowercase
    text = text.lower()

    # Pecah to array
    words = text.split(" ");

    # DF to Array
    arr_slang = df_tmp['slang'].to_numpy()
    arr_formal = df_tmp['formal'].to_numpy()
    do_not_change = df_slang_custom_exception['kata'].to_numpy()

    # Array New Text
    new_text = []

    for kata in words:
      if kata in arr_slang:
        if kata not in do_not_change:
          idx = np.where(arr_slang==kata)[0][0]
          kata = arr_formal[idx]

      new_text.append(kata)
      text_new = ' '.join(new_text)

    return text_new

<ipython-input-8-188833ac7026>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_slang_nasalsabila.drop_duplicates(inplace=True)


In [ ]:
## Load Preprocess Stemming Modul
# Load and Download Library Sastrawi
!pip install Sastrawi
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

# Indonesian Stopword by Sastrawi
factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()

# import StemmerFactory class
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 1.4 MB/s eta 0:00:00


In [ ]:
## Load Preprocess Stopword Data

# Load Data Stopword from Sastrawi
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()

# Load Data Stopword from Satya and Custom
stopword_satya = list(pd.read_csv('data/stopword/stopwords_satya.txt', header = None)[0])
custom_stopword = list(pd.read_csv('data/stopword/custom_stopword.txt', header = None)[0])

# Declare Function / Modul Stopword
def remove_stopwords(text):
  ## Stopword Removal from data by Sastrawi and Satwa
  text = stopword.remove(text)

  ## Stopword Removal from data by Satwa and Custom Word
  # Removal Stopword
  text_split = text.split(' ')
  temp_text_split = []
  for i in range(len(text_split)):
    ## Remove Stopword by Satwa : gak dipake dulu ini terlalu ngilangin konteks text
    #if (text_split[i] not in stopword_satya) and (text_split[i] not in custom_stopword):
    if (text_split[i] not in custom_stopword):
      temp_text_split.append(text_split[i])
  text = ' '.join(temp_text_split)

  return text

In [ ]:
def clean_text_predict(text) :

    ## TEXT TRANSFORM
    # Remove username 1st
    text = re.sub('@[^\s]+','',text)
    # Remove hashtag dan @
    ''.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",text).split())
    # Change "-" to space
    text = text.replace("-", " ")
    # Remove punctiation
    text  = "".join([char for char in text if char not in string.punctuation])
    # Remove angka
    text = re.sub('[0-9]+', '', text)
    # Remove website link
    text = re.sub(r'http\S+', '', text)
    # Remove new line
    text = re.sub("\n"," ",text)
    # Transform to lowercase
    text = text.lower()
    # Replace perulangan huruf pada kalimat "iyaaa" to "iya"
    text = re.sub(r'(\w)(\1{2,})', r"\1", text)
    # will replace single word
    text = re.sub(r"\b[a-zA-Z]\b","",text)
    # will replace space more than one
    text = re.sub('( {2,})',' ',text)

    ## SLANG
    text = remove_slang(text ,df_slang_nasalsabila)
    text = remove_slang(text ,df_slang_fendiirfan)
    text = remove_slang(text ,df_slang_custom)

    ## STEMMING
    text = stemmer.stem(text)

    ## STOPWORD
    text = remove_stopwords(text)

    ## TEXT TRANSFORM BUT AFTER
    # will replace single word
    # text = re.sub(r"\b[a-zA-Z]\b","",text)

    return text

In [ ]:
review_text = "Keren banget kolaborasinya jadi pengen beli."
txt_clean = clean_text_predict(review_text)

encoded_review = tokenizer.encode_plus(
    txt_clean,
    max_length=MAX_LEN,
    add_special_tokens=True,
    return_token_type_ids=False,
    pad_to_max_length=True,
    return_attention_mask=True,
    return_tensors='pt',
)
input_ids = encoded_review['input_ids'].to(device)
attention_mask = encoded_review['attention_mask'].to(device)

output = model(input_ids, attention_mask)
_, prediction = torch.max(output, dim=1)

print(f'Review text\t: {review_text}')
print(f'Clean text\t: {txt_clean}')
print(f'Sentiment\t: {class_names[prediction]}')

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2619: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Review text	: Keren banget kolaborasinya jadi pengen beli.
Clean text	: keren banget kolaborasi jadi pengin beli
Sentiment	: Positive


In [ ]:
def predict_sentiment(text) :
  # Preprocess Data Text
  ###############################################################################
  # Proses dibawah dikomen karena data masuk data yg udh preprocessing
  text = str(text)
  txt_clean = clean_text_predict(text)
  # Langsung Masuk Predict
  # txt_clean = text
  ###############################################################################

  # Set Parameter Input
  encoded_review = tokenizer.encode_plus(
      txt_clean,
      max_length=MAX_LEN,
      add_special_tokens=True,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt',
  )

  # Transform Data to Tensors
  input_ids = encoded_review['input_ids'].to(device)
  attention_mask = encoded_review['attention_mask'].to(device)

  # Predict
  output = model(input_ids, attention_mask)
  _, prediction = torch.max(output, dim=1)

  # Return Data
  return class_names[prediction]

In [ ]:
# df_mentionlytics_straykids = pd.read_excel('mentionlytics/straykids_ultramilk_mentionlytics.xlsx', index_col=0)
# df_mentionlytics_straykids = df_mentionlytics_straykids[['Content', 'Sentiment']]
# df_mentionlytics_straykids = df_mentionlytics_straykids.rename(columns={'Content': 'text', 'Sentiment': 'sentiment'})
# df_mentionlytics_straykids

In [ ]:
# df_mentionlytics_straykids['predicted'] = df_mentionlytics_straykids['text'].apply(lambda x: predict_sentiment(x))

In [ ]:
# df_mentionlytics_straykids

In [ ]:
# df_mentionlytics_straykids.to_excel('mentionlytics/straykids_ultramilk_mentionlytics_predicted.xlsx')

In [ ]:
# df_mentionlytics_itzy = pd.read_excel('mentionlytics/itzy_ultramilk_mentionlytics.xlsx', index_col=0)
# df_mentionlytics_itzy = df_mentionlytics_itzy[['Content', 'Sentiment']]
# df_mentionlytics_itzy = df_mentionlytics_itzy.rename(columns={'Content': 'text', 'Sentiment': 'sentiment'})
# df_mentionlytics_itzy

In [ ]:
# df_mentionlytics_itzy['predicted'] = df_mentionlytics_itzy['text'].apply(lambda x: predict_sentiment(x))

In [ ]:
# df_mentionlytics_itzy

In [ ]:
# df_mentionlytics_itzy.to_excel('mentionlytics/itzy_ultramilk_mentionlytics_predicted.xlsx')

In [ ]:
df_under_2022 = pd.read_excel('mentionlytics/all_data_under_2022.xlsx', index_col=0)
df_under_2022 = df_under_2022.drop(['tahun', 'lang', 'conf_score'], axis=1)
df_under_2022

<ipython-input-24-8d3933653843>:1: FutureWarning: Inferring datetime64[ns, UTC] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns, UTC])
  df_under_2022 = pd.read_excel('mentionlytics/all_data_under_2022.xlsx', index_col=0)


,date,platform,brand,hashtag,name,username,profile_id,comments,share_retweet,bookmark,quotes,likes,link,text
0,2021-08-30 00:00:00+00:00,Twitter,cimory,cimoryyogurt,makcellyahh🤭,@amikmooie,1253223231483343104,0,0,0,0,0,https://twitter.com/amikmooie/status/143231210...,Rasa banana ini lucu jugaak🤭🤭😚😚 #cimoryyogurt
1,2019-12-18 00:00:00+00:00,Twitter,cimory,cimoryyogurt,bukan siapa-siapa,@Nabilaftr2,1624285875923484928,0,0,0,0,1,https://twitter.com/Nabilaftr2/status/12072840...,Sayangi ususmu minum cimory tiap hari.. Ehh😂 #...
2,2017-08-14 00:00:00+00:00,Twitter,cimory,cimoryyogurt,Mumun Munawaroh,@mumunmanawaroh,860373179180605440,0,0,0,0,0,https://twitter.com/mumunmanawaroh/status/8971...,Yogurt dulu😉 #cimory #cimoryyogurt #enakbanget...
3,2017-07-04 00:00:00+00:00,Twitter,cimory,cimoryyogurt,wiThamee,@Withanissa,1177766821560143872,0,0,0,0,0,https://twitter.com/Withanissa/status/88224114...,..biar adem 😬 . . . . #yoghurt #cimoryyogurt #...
4,2017-03-01 00:00:00+00:00,Twitter,cimory,cimoryyogurt,Cimory Indonesia,@Cimory,730279728867737600,1,1,0,0,2,https://twitter.com/Cimory/status/836912161687...,Hai Cimories! Sekarang udah bulan Maret. Coba ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10258,2020-09-10 03:45:05+00:00,Instagram,milklife,#TheMilkofLife,GlobalFood,globalfoodid,17849789426318800,0,0,0,0,0,https://www.instagram.com/p/CE8OoRtAXud/,Pasti Seger nih susu nya 😁
10259,2020-09-10 03:40:28+00:00,Instagram,milklife,#TheMilkofLife,ZakirZavier | MASAK | KULINER,abangadeksukamam,17862431987101570,0,0,0,0,0,https://www.instagram.com/p/CE8OoRtAXud/,wah oke bener nih susunya
10260,2020-09-10 03:38:53+00:00,Instagram,milklife,#TheMilkofLife,Food+Lifestyle Blogger | Vera,verajj,17881808839761910,0,0,0,0,0,https://www.instagram.com/p/CE8OoRtAXud/,😍😍😍 rekomen nih susu nya
10261,2020-09-10 03:38:39+00:00,Instagram,milklife,#TheMilkofLife,STEVEN LUKITA,makansamakoko,17887995268678600,0,0,0,0,0,https://www.instagram.com/p/CE8OoRtAXud/,Sehat ya jamesss


In [ ]:
df_under_2022['predicted'] = df_under_2022['text'].apply(lambda x: predict_sentiment(x))

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2619: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
df_under_2022

,date,platform,brand,hashtag,name,username,profile_id,comments,share_retweet,bookmark,quotes,likes,link,text,predicted
0,2021-08-30 00:00:00+00:00,Twitter,cimory,cimoryyogurt,makcellyahh🤭,@amikmooie,1253223231483343104,0,0,0,0,0,https://twitter.com/amikmooie/status/143231210...,Rasa banana ini lucu jugaak🤭🤭😚😚 #cimoryyogurt,Positive
1,2019-12-18 00:00:00+00:00,Twitter,cimory,cimoryyogurt,bukan siapa-siapa,@Nabilaftr2,1624285875923484928,0,0,0,0,1,https://twitter.com/Nabilaftr2/status/12072840...,Sayangi ususmu minum cimory tiap hari.. Ehh😂 #...,Negative
2,2017-08-14 00:00:00+00:00,Twitter,cimory,cimoryyogurt,Mumun Munawaroh,@mumunmanawaroh,860373179180605440,0,0,0,0,0,https://twitter.com/mumunmanawaroh/status/8971...,Yogurt dulu😉 #cimory #cimoryyogurt #enakbanget...,Negative
3,2017-07-04 00:00:00+00:00,Twitter,cimory,cimoryyogurt,wiThamee,@Withanissa,1177766821560143872,0,0,0,0,0,https://twitter.com/Withanissa/status/88224114...,..biar adem 😬 . . . . #yoghurt #cimoryyogurt #...,Neutral
4,2017-03-01 00:00:00+00:00,Twitter,cimory,cimoryyogurt,Cimory Indonesia,@Cimory,730279728867737600,1,1,0,0,2,https://twitter.com/Cimory/status/836912161687...,Hai Cimories! Sekarang udah bulan Maret. Coba ...,Neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10258,2020-09-10 03:45:05+00:00,Instagram,milklife,#TheMilkofLife,GlobalFood,globalfoodid,17849789426318800,0,0,0,0,0,https://www.instagram.com/p/CE8OoRtAXud/,Pasti Seger nih susu nya 😁,Positive
10259,2020-09-10 03:40:28+00:00,Instagram,milklife,#TheMilkofLife,ZakirZavier | MASAK | KULINER,abangadeksukamam,17862431987101570,0,0,0,0,0,https://www.instagram.com/p/CE8OoRtAXud/,wah oke bener nih susunya,Neutral
10260,2020-09-10 03:38:53+00:00,Instagram,milklife,#TheMilkofLife,Food+Lifestyle Blogger | Vera,verajj,17881808839761910,0,0,0,0,0,https://www.instagram.com/p/CE8OoRtAXud/,😍😍😍 rekomen nih susu nya,Neutral
10261,2020-09-10 03:38:39+00:00,Instagram,milklife,#TheMilkofLife,STEVEN LUKITA,makansamakoko,17887995268678600,0,0,0,0,0,https://www.instagram.com/p/CE8OoRtAXud/,Sehat ya jamesss,Positive


In [ ]:
df_under_2022['date'] = df_under_2022['date'].dt.tz_localize(None)

In [ ]:
df_under_2022.to_excel('mentionlytics/all_data_under_2022_predicted.xlsx')

In [ ]:
# ## Declare Hashtag Data
# hashtag = "ultramilkxstraykids"

# ## Load Data Unpredicted
# df = pd.read_csv('data/training/'+hashtag+'/'+hashtag+'.csv')

# ## Drop All Rows with Polarity '4'
# df = df.drop(df[df['polarity'] == 4].index)

# ## Print DF
# print(df)

In [ ]:
# df['prediction'] = df['text_final'].apply(lambda x: predict_sentiment(x))

In [ ]:
# i = 0
# for index, row in df.iterrows():
#   if ((row['polarity'] == 0) and row['prediction'] == 'Negative') or ((row['polarity'] == 1) and row['prediction'] == 'Neutral')  or (((row['polarity'] == 2) or (row['polarity'] == 3)) and row['prediction'] == 'Positive') :
#     i = i + 1

# total_acc = ((i)/len(df)) * 100
# print(str(total_acc)+'%')

In [ ]:
# df.to_excel('report/'+hashtag+'/'+hashtag+'_all_predicted.xlsx')

# INI BUAT TAMBAHAN PROSES AJA TIDAK PERLU DIPAKAI

In [ ]:
# df_dupe = pd.read_excel('temp/dupe.xlsx')

In [ ]:
# df_dupe

In [ ]:
# df_dupe['text_final'] = df_dupe['text'].apply(lambda x: clean_text_predict(x))

In [ ]:
# df_dupe.to_excel('temp/dupe.xlsx')